In [34]:
import requests
import json
import prettytable
import pandas as pd
import os
import psycopg2
from dotenv import load_dotenv
from sqlalchemy import create_engine
import pprint


In [127]:
!pwd

/c/Users/aquir/Documents/Projects/ETL


In [35]:
headers = {'Content-type': 'application/json'}

# Convert dict into a JSON string
data = json.dumps({"seriesid": ['LNS14000000','LAUMT123310000000003'],"startyear":"2009", "endyear":"2019", "registrationkey":"43cd87b06a5e4f5799b72697bc1d5f9f"})

# Sending post request and saving response as response object (p)
p = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/', data=data, headers=headers)

# Create a python object 
json_data = json.loads(p.text)

# Pretty-printing a nested dictionary using "pprint" module:  prints the entries indented to match their nesting level.
pprint.pprint(json_data)

# Create a Dict with empty lists as values
result_dict = {'seriesID': [], 'periodname': [], 'year': [], 'period':[], 'value': []}

# Nested for loop to iterate through the json and append values to my previous dict
for series in json_data['Results']['series']:
        for item in series['data']:
            result_dict['seriesID'].append(series['seriesID'])
            result_dict['periodname'].append(item['periodName'])
            result_dict['year'].append(item['year'])
            result_dict['period'].append(item['period'])
            result_dict['value'].append(item['value'])
                                             

{'Results': {'series': [{'data': [{'footnotes': [{}],
                                   'latest': 'true',
                                   'period': 'M12',
                                   'periodName': 'December',
                                   'value': '3.5',
                                   'year': '2019'},
                                  {'footnotes': [{}],
                                   'period': 'M11',
                                   'periodName': 'November',
                                   'value': '3.5',
                                   'year': '2019'},
                                  {'footnotes': [{}],
                                   'period': 'M10',
                                   'periodName': 'October',
                                   'value': '3.6',
                                   'year': '2019'},
                                  {'footnotes': [{}],
                                   'period': 'M09',
                                

In [97]:
# Create a dataframe using above dict
result_dict_df = pd.DataFrame(result_dict)

# Preview result_dict_df 
result_dict_df.head(10)

,seriesID,periodname,year,period,value
0,LNS14000000,December,2019,M12,3.5
1,LNS14000000,November,2019,M11,3.5
2,LNS14000000,October,2019,M10,3.6
3,LNS14000000,September,2019,M09,3.5
4,LNS14000000,August,2019,M08,3.7
5,LNS14000000,July,2019,M07,3.7
6,LNS14000000,June,2019,M06,3.7
7,LNS14000000,May,2019,M05,3.6
8,LNS14000000,April,2019,M04,3.6
9,LNS14000000,March,2019,M03,3.8


In [119]:
# Create a dataframe for serieid: LNS14000000 - US
national_unemployment_rate_df = result_dict_df[result_dict_df['seriesID'] == 'LNS14000000']

# Adding a new column to identofy the region corectly
national_unemployment_rate_df['region'] = 'US'

# Adding a key (id) column 
national_unemployment_rate_df['id'] = range (1, 1+len(national_unemployment_rate_df))

# Moving id column to the first place
national_unemployment_rate_df= national_unemployment_rate_df[['id','seriesID','periodname', 'year', 'period', 'value', 'region']]




C:\Users\aquir\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\aquir\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [118]:
# Create a dataframe for serieid: LAUMT123310000000003 -  Miami-Fort Lauderdale-West Palm Beach, FL Metropolitan Statistical Area
Miami_unemployment_rate_df = result_dict_df[result_dict_df['seriesID'] == 'LAUMT123310000000003']

# Adding a new column to identify the region corectly
Miami_unemployment_rate_df['region'] = 'Miami-Fort Lauderdale-West Palm Beach'


# Adding a key (id) column 
Miami_unemployment_rate_df['id'] = range (1, 1+len(Miami_unemployment_rate_df))

# Convert superkey from int to str
Miami_unemployment_rate_df['id'] = Miami_unemployment_rate_df['id'].astype(int)

# Moving id column to the first place
Miami_unemployment_rate_df = Miami_unemployment_rate_df[['id','seriesID','periodname', 'year', 'period', 'value', 'region']]

C:\Users\aquir\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\aquir\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\aquir\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas

In [122]:
# Preview ational_unemployment_rate_df
national_unemployment_rate_df.head(10)


,id,seriesID,periodname,year,period,value,region
0,1,LNS14000000,December,2019,M12,3.5,US
1,2,LNS14000000,November,2019,M11,3.5,US
2,3,LNS14000000,October,2019,M10,3.6,US
3,4,LNS14000000,September,2019,M09,3.5,US
4,5,LNS14000000,August,2019,M08,3.7,US
5,6,LNS14000000,July,2019,M07,3.7,US
6,7,LNS14000000,June,2019,M06,3.7,US
7,8,LNS14000000,May,2019,M05,3.6,US
8,9,LNS14000000,April,2019,M04,3.6,US
9,10,LNS14000000,March,2019,M03,3.8,US


In [123]:
# Preview Miami_unemployment_rate_df
Miami_unemployment_rate_df.head(10)


,id,seriesID,periodname,year,period,value,region
132,1,LAUMT123310000000003,November,2019,M11,2.7,Miami-Fort Lauderdale-West Palm Beach
133,2,LAUMT123310000000003,October,2019,M10,2.9,Miami-Fort Lauderdale-West Palm Beach
134,3,LAUMT123310000000003,September,2019,M09,3.0,Miami-Fort Lauderdale-West Palm Beach
135,4,LAUMT123310000000003,August,2019,M08,3.5,Miami-Fort Lauderdale-West Palm Beach
136,5,LAUMT123310000000003,July,2019,M07,3.4,Miami-Fort Lauderdale-West Palm Beach
137,6,LAUMT123310000000003,June,2019,M06,3.3,Miami-Fort Lauderdale-West Palm Beach
138,7,LAUMT123310000000003,May,2019,M05,3.1,Miami-Fort Lauderdale-West Palm Beach
139,8,LAUMT123310000000003,April,2019,M04,2.9,Miami-Fort Lauderdale-West Palm Beach
140,9,LAUMT123310000000003,March,2019,M03,3.3,Miami-Fort Lauderdale-West Palm Beach
141,10,LAUMT123310000000003,February,2019,M02,3.3,Miami-Fort Lauderdale-West Palm Beach


In [124]:
# Load dotenv
load_dotenv()

# Create Engine - Accessing the Environment Variables
engine = create_engine(f"postgresql://{os.environ.get('DB_USERNAME')}:{os.environ.get('DB_PASSWORD')}@{os.environ.get('HOST')}/{os.environ.get('DB_NAME')}")


In [100]:
# Check for tables
engine.table_names()

['miami_unemployment', 'national_unemployment']

In [ ]:
# Use pandas to load json 'Region: US' converted DataFrame into database
national_unemployment_rate_df.to_sql(name='national_unemployment', con=engine, if_exists='append', index=False)


In [126]:
# Confirm data has been added by querying the customer_name table
pd.read_sql_query('select * from national_unemployment', con=engine).head()

,id,seriesID,periodname,year,period,value,region
0,1,LNS14000000,December,2019,M12,3.5,US
1,2,LNS14000000,November,2019,M11,3.5,US
2,3,LNS14000000,October,2019,M10,3.6,US
3,4,LNS14000000,September,2019,M09,3.5,US
4,5,LNS14000000,August,2019,M08,3.7,US


In [ ]:
# Use pandas to load json 'Region: Miami-Fort Lauderdale-West Palm Beach' converted DataFrame into database
Miami_unemployment_rate_df.to_sql(name='miami_unemployment', con=engine, if_exists='append', index=False)

In [116]:
# Confirm data has been added by querying the customer_name table
pd.read_sql_query('select * from miami_unemployment', con=engine).head()

,id,seriesID,periodname,year,period,value,region
0,1,LAUMT123310000000003,November,2019,M11,2.7,Miami-Fort Lauderdale-West Palm Beach
1,2,LAUMT123310000000003,October,2019,M10,2.9,Miami-Fort Lauderdale-West Palm Beach
2,3,LAUMT123310000000003,September,2019,M09,3.0,Miami-Fort Lauderdale-West Palm Beach
3,4,LAUMT123310000000003,August,2019,M08,3.5,Miami-Fort Lauderdale-West Palm Beach
4,5,LAUMT123310000000003,July,2019,M07,3.4,Miami-Fort Lauderdale-West Palm Beach
